<a href="https://colab.research.google.com/github/deepdarkssj/diff-svc/blob/main/diff-svc%E4%B8%80%E9%94%AE%E5%90%88%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 写在前面

本笔记本用于使用[diff-svc歌声转换模型](https://github.com/prophesier/diff-svc)，将任意歌声转换为[永雏塔菲](https://space.bilibili.com/1265680561)音色

**你需要准备一段不含背景音乐的纯干声，不宜有过重的混响或明显和声，若你的音频经过类似去伴奏的处理，请尽量使用降噪、人声增强、去混响等方式保证音频质量。音频格式为wav，采样率不得低于16KHZ**


**⬇ 使用本模型即代表您理解并同意diff-svc的注意事项 ⬇**

*本项目是基于学术交流目的建立，并非为生产环境准备，不对由此项目模型产生的任何声音的版权问题负责。
如将本仓库代码二次分发，或将由此项目产出的任何结果公开发表(包括但不限于视频网站投稿)，请注明原作者及代码来源(此仓库)。
如果将此项目用于任何其他企划，请提前联系并告知本仓库作者,十分感谢。*

[永雏塔菲歌声转换模型(AI塔菲2.0)介绍](https://www.bilibili.com/video/BV1144y1Q7w4/)

[本笔记本作者@极黑的绅士君](https://space.bilibili.com/605886)

[diff-svc作者@14-26AA](https://space.bilibili.com/485443)

[永雏塔菲QQ频道](https://pd.qq.com/s/9gcb5eo0a)

**请按照本笔记本中的步骤自上而下依次执行，若不慎对代码或运行时进行了不可挽回的修改导致报错，请点击上方[代码执行程序]-[断开连接并删除运行时]从而初始化环境并重新开始。在使用中发现任何问题欢迎点击上方频道链接加入塔频讨论**

**现在，试着将鼠标移到下方[配置环境]中的第一个代码块里，点击左上角的播放(其实是执行)按钮吧！**


# 配置环境

In [ ]:
#本单元格约需执行2分钟
%cd /content
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install --upgrade pip
!git clone -b main https://github.com/deepdarkssj/diff-svc
%cd diff-svc
!mkdir ./checkpoints/hubert
!mkdir raw
!mkdir results
!pip install -r requirements.txt

In [ ]:
#本单元格约需执行5秒
import io
import time
from pathlib import Path

import librosa
import numpy as np
import soundfile

from infer_tools import infer_tool
from infer_tools import slicer
from infer_tools.infer_tool import Svc
from utils.hparams import hparams
chunks_dict = infer_tool.read_temp("./infer_tools/chunks_temp.json")

## 从Google Drive下载模型

In [ ]:
#本单元格约需执行30秒
!pip install --upgrade --no-cache-dir gdown
!gdown --id '1Ju4wIViS7w7HSkeV4qW84zSXYYWtuuot' --output /content/diff-svc/checkpoints/0102_xiaoma_pe/model_ckpt_steps_60000.ckpt
!gdown --id '1kZM9hx5c--nqIkuIVSSXKnRUCkL0FjjR' --output /content/diff-svc/checkpoints/hubert/hubert_soft.pt
!gdown --id '1Q5oS7L6vQ1w-g059snFUu8TWX-0M6-cm' --output /content/diff-svc/checkpoints/0109_hifigan_bigpopcs_hop128/model_ckpt_steps_1512000.ckpt
!gdown --id '1zqdZMKRXJt72HzR9Yw-dK2-OfW0f8hPx' --output /content/diff-svc/checkpoints/0109_hifigan_bigpopcs_hop128/model_ckpt_steps_1512000.pth
!gdown --id '1yqEekFgO6vZ12XZgwdkKrSeAwsVRub66' --output /content/diff-svc/checkpoints/taffy/model_ckpt_steps_210000.ckpt



## 上传源音频

请点击左侧文件夹按钮打开运行时内的文件目录，将你准备的人声传入至raw文件夹，直接拖入或点击上传按钮均可

**注意：运行时内的文件在关闭网页后会被删除！**

因此你也许需要使用Google Drive来存储一些数据，下面的单元格会将你的Google Drive与左侧文件目录关联

In [ ]:
#@title （可选）关联Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 声音转换

终于到了最关键的一步！

**按需填写代码单元格右侧的设定，按下执行按钮就好啦！由于音频时长不同，合成所需的时长也不同，耐心等待吧！**

**结果会自动输出至results文件夹。**

In [ ]:
import infer

#@markdown **音高调整（升降n个半音）**
vc_transform = "0" #@param {type:"string"}
#@markdown 人声文件名（不带.wav）
clean_name = "jineng" #@param {type:"string"}

project_name = "taffy"
model_path = f'./checkpoints/{project_name}/model_ckpt_steps_210000.ckpt'
config_path = f'./checkpoints/{project_name}/config.yaml'
file_names = [clean_name]
trans = [int(vc_transform)]
accelerate = 20
hubert_gpu = True
format='flac'
step = int(model_path.split("_")[-1].split(".")[0])
infer_tool.mkdir(["./raw", "./results"])
infer_tool.fill_a_to_b(trans, file_names)
model = Svc(project_name, config_path, hubert_gpu, model_path)
for f_name, tran in zip(file_names, trans):
    if "." not in f_name:
        f_name += ".wav"
    source_path = infer.run_clip(model, key=tran, acc=accelerate, use_crepe=True, thre=0.05, use_pe=True, use_gt_mel=False,
             add_noise_step=500, f_name=f_name, project_name=project_name, format=format, step=step)


**恭喜大功告成！你可以查看results文件夹，右键下载**

**执行下面的单元格可以让你在线预览输出的结果(还是要缓冲的，只是不用下载到本地罢了)**

In [ ]:
import IPython.display as ipd
import torchaudio
audio,sr = torchaudio.load(source_path)
ipd.display(ipd.Audio(audio, rate=sr))